In [272]:
import pandas as pd
import sys
sys.path.insert(1, r'C:\Users\adars\OneDrive\Escritorio\Uni\datathon\healthcare_challenge')
from utils import *
import pickle
# catboosting
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [273]:
df = pd.read_csv('../data/consumo_material_clean_with_category.csv', parse_dates=['FECHAPEDIDO'])
# df_train = pd.read_csv('../data/consumo_no23.csv')
# df_test = pd.read_csv('../data/consumo_23.csv')

df_transito = df[df['TGL'] == 'TRANSITO']
df_alma = df[df['TGL'] == 'ALMACENABLE']

specified_ids = ['E65159', 'E64751', 'E64764']
df_specified_ids = df_alma[df_alma['CODIGO'].isin(specified_ids)]
df_other_ids = df_alma[~df_alma['CODIGO'].isin(specified_ids)]
df_other_ids_gt_10 = df_other_ids[df_other_ids['UNIDADESCONSUMOCONTENIDAS'] > 10]
df_other_ids_lt_10 = df_other_ids[df_other_ids['UNIDADESCONSUMOCONTENIDAS'] < 10]

In [274]:
df_transito.shape

(4035, 13)

In [196]:
df_other_ids[df_other_ids['STACKS_COMPRATS'] > 100]['UNIDADESCONSUMOCONTENIDAS'].unique()

array([ 3, 10,  5], dtype=int64)

In [197]:
df_other_ids[df_other_ids['STACKS_COMPRATS'] < 100]['UNIDADESCONSUMOCONTENIDAS'].unique()

array([ 10,  50, 300, 100,   5,  40,  80,   3, 240,  70,  30, 350,  60,
       480, 560, 160, 200, 500, 180, 140], dtype=int64)

In [275]:
df = preprocess_cosumo(df_transito)
df.head()

asjfkjdsfljds


C:\Users\adars\OneDrive\Escritorio\Uni\datathon\healthcare_challenge\utils.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['STACKS_COMPRATS'] = data['CANTIDADCOMPRA']/data['UNIDADESCONSUMOCONTENIDAS']
C:\Users\adars\OneDrive\Escritorio\Uni\datathon\healthcare_challenge\utils.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['STACKS_COMPRATS'] = data['STACKS_COMPRATS'].apply(lambda x: int(x))
C:\Users\adars\OneDrive\Escritorio\Uni\datathon\healthcare_challenge\utils.py:12: SettingWithCopyWarn

,FECHAPEDIDO,UNIDADESCONSUMOCONTENIDAS,PRECIO,STACKS_COMPRATS,CODIGO_B40558,CODIGO_E64543,CODIGO_E64761,CODIGO_E64898,CODIGO_E78950,CODIGO_E99808,...,CODIGO_F43331,CODIGO_F43580,CODIGO_F43581,CODIGO_F43585,CODIGO_F44200,CODIGO_F46843,CODIGO_F46846,CODIGO_F50071,TIPOCOMPRA_Compra menor,TIPOCOMPRA_Concurso
0,2023-01-01,10,62.590,6,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,2016-01-02,5,12.100,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,2018-01-02,5,12.100,8,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
10,2018-01-02,350,577.500,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
14,2019-01-02,5,13.497,10,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [276]:
df['trimester'] = df['FECHAPEDIDO'].dt.quarter
#df['month'] = df['FECHAPEDIDO'].dt.month

In [277]:
grouped_df = df.groupby(['trimester', df['FECHAPEDIDO'].dt.year])

In [278]:
len(grouped_df)

36

In [279]:
trimester_dfs = [grouped_df.get_group((trimester, year)).reset_index(drop=True) for (trimester, year), _ in grouped_df]
dfs_chunked = [trimester_dfs[i:i + 9] for i in range(0, len(trimester_dfs), 9)]
result_dfs = []
for chunk in dfs_chunked:
    result_dfs.append(pd.concat(chunk, axis=0, ignore_index=True))

In [280]:
len(result_dfs)

4

In [261]:
#trimester_1_df = result_dfs[0]
#trimester_2_df = result_dfs[1]
#trimester_3_df = result_dfs[2]
#trimester_4_df = result_dfs[3]

In [281]:
target = result_dfs[0]

In [282]:
target.drop(['trimester'], axis = 1, inplace = True)
target.head()

,FECHAPEDIDO,UNIDADESCONSUMOCONTENIDAS,PRECIO,STACKS_COMPRATS,CODIGO_B40558,CODIGO_E64543,CODIGO_E64761,CODIGO_E64898,CODIGO_E78950,CODIGO_E99808,...,CODIGO_F43331,CODIGO_F43580,CODIGO_F43581,CODIGO_F43585,CODIGO_F44200,CODIGO_F46843,CODIGO_F46846,CODIGO_F50071,TIPOCOMPRA_Compra menor,TIPOCOMPRA_Concurso
0,2015-02-02,5,12.10,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2015-02-03,350,611.66,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2015-02-09,5,12.10,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2015-03-03,350,611.66,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2015-03-06,5,12.10,6,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [283]:
df_train = target[target['FECHAPEDIDO']<dt.datetime(year=2023,month=1,day=1)]
#df_train = df_train[df_train['FECHAPEDIDO'] >= dt.datetime(year=2021,month=1,day=1)]
df_test = target[target['FECHAPEDIDO']>=dt.datetime(year=2023,month=1,day=1)]

df_train.drop(columns=['FECHAPEDIDO'], inplace=True)
df_test.drop(columns=['FECHAPEDIDO'], inplace=True)

C:\Users\adars\AppData\Local\Temp\ipykernel_16344\1363158763.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(columns=['FECHAPEDIDO'], inplace=True)
C:\Users\adars\AppData\Local\Temp\ipykernel_16344\1363158763.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(columns=['FECHAPEDIDO'], inplace=True)


In [284]:
print(df_train.shape)
print(df_test.shape)

(648, 21)
(287, 21)


In [285]:
X = df_train.drop(['STACKS_COMPRATS'], axis = 1)
y = df_train['STACKS_COMPRATS']
X_test = df_test.drop(['STACKS_COMPRATS'], axis = 1)
y_test = df_test['STACKS_COMPRATS'].reset_index(drop=True)

In [286]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', subsample = 1, reg_lambda = 0.2, reg_alpha = 0.1, colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 500)
xg_reg.fit(X, y)

[01:04:56] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', ...)

In [287]:
# Predict the inventory units for the test and train sets using the trained CatBoostRegressor model 'gsc1'
y_predicted_test = xg_reg.predict(X_test)
y_predicted_train = xg_reg.predict(X)

# Calculate the root mean squared error (RMSE) between the predicted and actual inventory units for the test and train sets
# The 'squared' parameter is set to False to return the RMSE instead of the MSE
rms_test = mean_squared_error(y_test, y_predicted_test)
rms_train = mean_squared_error(y, y_predicted_train)

# Print the RMSE values for the test and train sets
print(f'train: {rms_train}\ntest: {rms_test}')

train: 14.34195058681892
test: 24.105272524594014


In [97]:
gsc1 = CatBoostRegressor(iterations = 500,
                            loss_function='RMSE',
                            learning_rate = 0.25,
                            depth = 3,
                            l2_leaf_reg = 0.2,
                            model_size_reg = 4
                            )
gsc1.fit(X, y)

0:	learn: 37.4473015	total: 138ms	remaining: 1m 8s
1:	learn: 32.5205686	total: 139ms	remaining: 34.7s
2:	learn: 30.1846132	total: 140ms	remaining: 23.3s
3:	learn: 27.7892176	total: 141ms	remaining: 17.5s
4:	learn: 25.4252032	total: 142ms	remaining: 14.1s
5:	learn: 23.9923565	total: 143ms	remaining: 11.8s
6:	learn: 23.2129796	total: 144ms	remaining: 10.1s
7:	learn: 22.3582675	total: 145ms	remaining: 8.89s
8:	learn: 21.5330888	total: 146ms	remaining: 7.94s
9:	learn: 21.2429362	total: 146ms	remaining: 7.17s
10:	learn: 20.0415258	total: 148ms	remaining: 6.57s
11:	learn: 19.2167120	total: 149ms	remaining: 6.05s
12:	learn: 18.6950489	total: 150ms	remaining: 5.63s
13:	learn: 18.3692737	total: 152ms	remaining: 5.26s
14:	learn: 18.0664197	total: 153ms	remaining: 4.93s
15:	learn: 17.8988587	total: 154ms	remaining: 4.66s
16:	learn: 17.3551957	total: 156ms	remaining: 4.44s
17:	learn: 17.1885106	total: 159ms	remaining: 4.27s
18:	learn: 17.0722691	total: 161ms	remaining: 4.09s
19:	learn: 16.7350837	

In [115]:
# Predict the inventory units for the test and train sets using the trained CatBoostRegressor model 'gsc1'
y_predicted_test = gsc1.predict(X_test)
y_predicted_train = gsc1.predict(X)

# Calculate the root mean squared error (RMSE) between the predicted and actual inventory units for the test and train sets
# The 'squared' parameter is set to False to return the RMSE instead of the MSE
rms_test = mean_squared_error(y_test, y_predicted_test)
rms_train = mean_squared_error(y, y_predicted_train)

# Print the RMSE values for the test and train sets
print(f'train: {rms_train}\ntest: {rms_test}')

train: 86.27987240807617
test: 969.5560103398387
